In [34]:
import requests
from bs4 import BeautifulSoup
import datetime

In [76]:
Dollar_rub = 'https://www.google.ru/search?newwindow=1&sxsrf=ALeKk00CGIqsvYlKeAslskUdBGGt11rFIA%3A1605979081454&source=hp&ei=yUu5X6aSGai5gwf9pLOYCw&q=dollar+rub&oq=dollar+rub&gs_lcp=CgZwc3ktYWIQAzIFCAAQywEyBQgAEMsBMgUIABDLATIFCAAQywEyBQgAEMsBMgIIADIFCAAQywEyBQgAEMsBMgUIABDLATIFCAAQywE6BAgjECc6BQgAELEDOggIABCxAxCDAToFCC4QsQM6BwgAEAoQywFQkAZY5idgwCloAHAAeACAAUuIAZsFkgECMTCYAQCgAQGqAQdnd3Mtd2l6&sclient=psy-ab&ved=0ahUKEwjmmqilkpTtAhWo3OAKHX3SDLMQ4dUDCAc&uact=5'
Euro_rub = 'https://www.google.ru/search?newwindow=1&sxsrf=ALeKk00mr36I6KAV8ahHu38LzCKOKqFNtg%3A1605979087533&ei=z0u5X4CPIMSQrgS8qZaQBQ&q=euro+rub&oq=euro+rub&gs_lcp=CgZwc3ktYWIQAzIHCAAQsQMQQzIFCAAQywEyBQgAEMsBMgUIABDLATIFCAAQywEyBAgAEEMyBQgAEMsBMgUIABDLATIECAAQCjIHCAAQChDLAToECAAQRzoGCAAQBxAeOggIABAHEAoQHjoCCABQ9LABWPS4AWD9vAFoAHADeACAAVmIAa0CkgEBNJgBAKABAaoBB2d3cy13aXrIAQjAAQE&sclient=psy-ab&ved=0ahUKEwjAsp2okpTtAhVEiIsKHbyUBVIQ4dUDCA0&uact=5'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'}

## курс доллара
full_page = requests.get(Dollar_rub, headers=headers )
soup = BeautifulSoup(full_page.content, 'html.parser')
convert = soup.findAll("span", {"class":"DFlfde", "class": "SwHCTb"})
rub_dollar = convert[0].text.replace(",",".")
## курс евро
full_page = requests.get(Euro_rub, headers=headers )
soup = BeautifulSoup(full_page.content, 'html.parser')
convert = soup.findAll("span", {"class":"DFlfde", "class": "SwHCTb"})
rub_euro = convert[0].text.replace(",",".")

#время и дата
full_page = requests.get(Euro_rub, headers=headers )
soup = BeautifulSoup(full_page.content, 'html.parser')
convert = soup.findAll("div", {"class":"hqAUc"})
print('Курс доллар по курсу ЦБ на ' + convert[0].text[0:20] + ": " + rub_dollar)
print('Курс евро по курсу ЦБ на ' + convert[0].text[0:20] + ": " + rub_euro)


Курс доллар по курсу ЦБ на 21 нояб., 14:43 UTC : 76.21
Курс евро по курсу ЦБ на 21 нояб., 14:43 UTC : 90.36
